In [4]:
import requests
import rasterio
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Define the bounding box for Stockton area
min_lon, min_lat = -121.4091, 37.9339
max_lon, max_lat = -121.2441, 38.0241

# Define the spatial reference and tile size
bboxSR = 4326  # WGS 84
imageSR = 3857  # Web Mercator
tile_size = 400  # Adjusted tile size for larger area

# Calculate the number of tiles
x_range = max_lon - min_lon
y_range = max_lat - min_lat

# Determine the resolution of each tile (adjusting to cover a larger area)
tile_resolution_x = x_range / 30  # Divide the area into a 10x10 grid (adjust as needed)
tile_resolution_y = y_range / 30

# Create a directory to save tiles
os.makedirs("stockton_tiles", exist_ok=True)

count = 0

# Loop over the area and fetch tiles
for x in np.arange(min_lon, max_lon, tile_resolution_x):
    for y in np.arange(min_lat, max_lat, tile_resolution_y):
        # Define the bounding box for the current tile
        bbox = f"{x},{y},{x+tile_resolution_x},{y+tile_resolution_y}"
        
        # Define the parameters for the API call
        url = "https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2020/ImageServer/exportImage"
        params = {
            "bbox": bbox,
            "bboxSR": bboxSR,
            "size": f"{tile_size},{tile_size}",
            "imageSR": imageSR,
            "format": "tiff",
            "f": "json"
        }

        # Make the API call with SSL verification disabled
        response = requests.get(url, params=params, verify=False)
        response_json = response.json()

        # Retrieve the image URL from the response
        if 'href' in response_json:
            image_url = response_json['href']
            image_response = requests.get(image_url, verify=False)

            if image_response.status_code == 200:
                tile_filename = f"stockton_tiles/tile_{x}_{y}.tiff"
                with open(tile_filename, 'wb') as file:
                    file.write(image_response.content)
                
                count += 1
                print(f"Saved tile at ({x}, {y}) to {tile_filename} Count: {count}")
                # Optionally read and display the image
                # with MemoryFile(image_response.content) as memfile:
                #     with memfile.open() as dataset:
                #         # Read the image
                #         image = dataset.read([1, 2, 3])  # Read the first three bands (R, G, B)

                #         plt.figure(figsize=(6, 6))
                #         plt.imshow(image.transpose(1, 2, 0))  # Transpose to match the (row, col, band) format
                #         plt.title(f'Tile at ({x}, {y})')
                #         plt.show()
            else:
                print(f"Failed to retrieve image data for tile at ({x}, {y}):", image_response.status_code)
        else:
            print(f"Failed to retrieve image URL for tile at ({x}, {y}):", response_json)